In [10]:
# optical flow with corner detection

import numpy as np
import cv2

cap = cv2.VideoCapture(0)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)


while True:
 ret, frame = cap.read()
 frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

 # calculate optical flow
 p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

 # Select good points
 if p1 is not None:
    good_new = p1[st==1]
    good_old = p0[st==1]

 # draw the tracks
 for i, (new, old) in enumerate(zip(good_new, good_old)):
    x_new, y_new = new.ravel()
    x_old, y_old = old.ravel()
    mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), color[i].tolist(), 2)
    frame = cv2.circle(frame, (int(x_new), int(y_new)), 5, color[i].tolist(), -1)

 img = cv2.add(frame, mask)

 cv2.imshow('frame', img)
 #cv2.imshow('mask', mask)

 k = cv2.waitKey(30) & 0xff
 if k == 27:
    break
 
 # Now update the previous frame and previous points
 old_gray = frame_gray.copy()
 p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()

In [40]:
import cv2
import numpy as np

points = []
k = False 
def onMouse(event, x, y, flag, param):
	global points,k
	if event == cv2.EVENT_LBUTTONDOWN:
		points.append((x, y)) 
		k = True
		

cv2.namedWindow("window")
cv2.setMouseCallback("window", onMouse)

cap = cv2.VideoCapture(0)

while True:
	ret, frame = cap.read()

	cv2.imshow("window", frame)

	key = cv2.waitKey(1) & 0xff
	if k:
		old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		cv2.destroyAllWindows()
		break

old_pts = np.array(points[0], dtype="float32").reshape(-1,1,2)
mask = np.zeros_like(frame)

while True:
	_, frame2 = cap.read()

	new_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

	new_pts,status,err = cv2.calcOpticalFlowPyrLK(old_gray, 
                         new_gray, 
                         old_pts, 
                         None, maxLevel=1,
                         criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 15, 0.08))

	# mask = cv2.line(mask, (int(new_pts.ravel()[0]), int(new_pts.ravel()[1])), (int(old_pts.ravel()[0]), int(old_pts.ravel()[1])), (36,255,100), 2)
	# frame2 = cv2.circle(frame2, (int(new_pts.ravel()[0]) ,int(new_pts.ravel()[1])), 2, (36,255,100), 2)
	cv2.circle(mask, (int(new_pts.ravel()[0]) ,int(new_pts.ravel()[1])), 2, (0,255,0), 2)
	combined = cv2.addWeighted(frame2, 0.7, mask, 0.3, 0.1)

	cv2.imshow("new win", mask)
	cv2.imshow("wind", combined)

	old_gray = new_gray.copy()
	old_pts = new_pts.copy()

	key = cv2.waitKey(1) & 0xff
	if key == 27:
		cap.release()
		cv2.destroyAllWindows()
		break 



**Dense Optical Flow**

In [53]:
# Dense Optical Flow

import numpy as np
import cv2

cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()
prev = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:, :, 1] = 255

while True:
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)        # convert cartesian coordinates of vector to polar coordinates

    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame2', bgr)
    key = cv2.waitKey(30) & 0xff
    if key == 27:
        break
    
    prev = next

cap.release()
cv2.destroyAllWindows()